<h1>Imports</h1>

In [4]:
from model_nn.net import net
from model_nn.create_model import create_model, load_checkpoint
from model_nn.trains import train, perf_measure, find_classes
from Random_forest_baseline.loader.loader import train_transforms_prepare, test_transforms_prepare
import torch
from torchvision import datasets
import matplotlib.pyplot as plt

<h1>Loading data</h1>

In [5]:
data_dir = 'Splited_dataset'
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'

train_transforms = train_transforms_prepare(size=(256,256), normalize_mean=(0,0,0), normalize_std=(1,1,1))
test_transforms = test_transforms_prepare(size=(256,256), center_crop=236, normalize_mean=(0,0,0), normalize_std=(1,1,1))

train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

<h1>Creating models</h1>

In [3]:
TYPE = "resnet"
NUM_LABEL = 48
OPTIM = "sgd"
LR = 0.01
MOMENTUM = 0.9
PATIENCE = 3
THRESHOLD = 0.9
EPOCH = 10

net1 = net(pre_type=TYPE, num_label=NUM_LABEL)

criterion, optimizer, lrscheduler = create_model(model=net1,
                                                 opti=OPTIM,
                                                 lr=LR,
                                                 momentum=MOMENTUM,
                                                 patience=PATIENCE,
                                                 threshold=THRESHOLD)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

<h1>Training</h1>

In [4]:
trained = train(net1, trainloader, testloader, optimizer, criterion, lrscheduler, EPOCH)

No. epochs: 2,                 Training Loss: 0.478                 Valid Loss: 1.126                 Valid Accuracy: 0.747
No. epochs: 4,                 Training Loss: 0.018                 Valid Loss: 0.633                 Valid Accuracy: 0.861
No. epochs: 5,                 Training Loss: 0.029                 Valid Loss: 0.415                 Valid Accuracy: 0.903
No. epochs: 7,                 Training Loss: 0.005                 Valid Loss: 0.385                 Valid Accuracy: 0.905
No. epochs: 8,                 Training Loss: 0.012                 Valid Loss: 0.323                 Valid Accuracy: 0.919
No. epochs: 10,                 Training Loss: 0.004                 Valid Loss: 0.314                 Valid Accuracy: 0.915


<h1>Save model</h1>

In [7]:
checkpoint = {'state_dict': trained.state_dict(),
              'model': trained.fc,
              'class_to_idx': train_data.class_to_idx,
              'opt_state': optimizer.state_dict,
              'num_epochs': EPOCH}

torch.save(checkpoint, 'model_nn/cp.pth')